In [ ]:
import numpy as np
import pickle, copy

import re, string
from nltk.corpus import stopwords
import pandas as pd
import argparse
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from nltk.util import ngrams
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
nltk.download('punkt')

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--data", required=True, help="path of the directory containing data files")

args = vars(parser.parse_args())
data_dir = args["data"]

In [ ]:
with open(data_dir + '/train', 'rb') as F:
    train = pickle.load(F)

with open(data_dir + '/test', 'rb') as F:
    test = pickle.load(F)

TweetInfoDF = pd.read_csv(data_dir + '/TweetInfoDF.csv')
TweetInfoDFText = list(TweetInfoDF['text'])

In [ ]:
def remove_punctuations_and_numbers(String):
    L = []
    for s in tokenize(String):
        if(s not in string.punctuation)and not(s>="0" and s<="9")  and not(s=="…") and 'a'<=s and s<='z':
            L.append(s)
    return " ".join(L)

def tokenize(String):
    return nltk.word_tokenize(String)

def remove_stopwords(List):
    L = []
    for s in List:
        if(s not in stopwords.words("english") and (s not in hindi_stp) and (s not in hinglish_stp)):
            L.append(s)
    return L

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [ ]:
cleaned_tweets = []

for i,j in train.iterrows():
    tw = TweetInfoDFText[i].lower()
    tw = strip_links(strip_all_entities(tw))
    tw = remove_punctuations_and_numbers(tw)
    cleaned_tweets.append(tw)

train_cleaned_tweets = copy.deepcopy(cleaned_tweets)

cleaned_tweets = []

for i,j in test.iterrows():
    tw = TweetInfoDFText[i].lower()
    tw = strip_links(strip_all_entities(tw))
    tw = remove_punctuations_and_numbers(tw)
    cleaned_tweets.append(tw)

test_cleaned_tweets = copy.deepcopy(cleaned_tweets)

cleaned_tweets = []
cleaned_tweets = test_cleaned_tweets
cleaned_tweets.extend(train_cleaned_tweets)

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    stop_words='english',
    ngram_range=(1, 3),
    max_features=30000)
word_vectorizer.fit(cleaned_tweets)
dataset_word_features = word_vectorizer.transform(cleaned_tweets)
print(dataset_word_features.shape)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 4),
    max_features=30000)
char_vectorizer.fit(cleaned_tweets)
dataset_char_features = char_vectorizer.transform(cleaned_tweets)
print(dataset_char_features.shape)

dataset_features = hstack([dataset_char_features, dataset_word_features])

dataset_features = dataset_features.todense()
X_train=[]
X_test=[]
y_train=[]
y_test=[]

index_ = 0
for i,j in test.iterrows():
    X_test.append(dataset_features[index_])
    y_test.append(j['Tag'])
    index_ = index_+1

for i,j in train.iterrows():
    X_train.append(dataset_features[index_])
    y_train.append(j['Tag'])
    index_ = index_+1


X_train = np.asarray(X_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[-1])
X_test = np.asarray(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[-1])  
y_train = y_train
y_test = y_test


In [ ]:
clf = svm.SVC(kernel='linear', class_weight='balanced')
clf.fit(X_train, y_train)
print(clf)
pred_y_test = clf.predict(X_test)
print(classification_report(y_test, pred_y_test))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.77      0.99      0.87      2285
           1       1.00      0.07      0.13        73
           2       0.89      0.55      0.68       917
           3       0.00      0.00      0.00        62
           4       1.00      0.09      0.17       169

    accuracy                           0.79      3506
   macro avg       0.73      0.34      0.37      3506
weighted avg       0.81      0.79      0.75      3506

